# Fake news detection using neural networks and linear regression

In [ ]:
#ide menjen az osszes package
!pip install contractions


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.9 MB/s eta 0:00:00


# Downloading dataset
The dataset: [GonzaloA/fake_news](https://huggingface.co/datasets/GonzaloA/fake_news)

In [ ]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/GonzaloA/fake_news/" + splits["train"])

df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,Unnamed: 0,title,text,label
0,0,‘Maury’ Show Official Facebook Posts F*CKED U...,Maury is perhaps one of the trashiest shows on...,0
1,1,Trump’s Favorite News Channel Tries To Soothe...,"Yesterday, after the father of one of the UCLA...",0
2,2,"Russia warns Iraq, Kurds not to destabilize Mi...",MOSCOW (Reuters) - Russia on Wednesday warned ...,1
3,3,WATCH STEVE SCALISE Throw A Strike At The Nati...,"House Majority Whip Steve Scalise (R., La.) th...",0
4,4,Trump Will HATE What Stephen Colbert Just Did...,It can be said that Late Show host Stephen Col...,0


# Text preprocessing
Removing stopwords, html tags and special characters.

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

def clean_text_advanced(text):
  text = text.lower()
  text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
  text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
  text = text.strip()
  text = BeautifulSoup(text, "html.parser").get_text() # Remove HTML tags
  text = re.sub(r"[<>]", "", text)
  stop_words = set(stopwords.words('english'))
  words = text.split()
  filtered_words = [word for word in words if word not in stop_words]
  text = " ".join(filtered_words)
  text = text.lower().strip()
  return text

df['text'] = df['text'].apply(clean_text_advanced)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Unnamed: 0,title,text,label
0,0,‘Maury’ Show Official Facebook Posts F*CKED U...,maury perhaps one trashiest shows television t...,0
1,1,Trump’s Favorite News Channel Tries To Soothe...,yesterday father one ucla players arrested chi...,0
2,2,"Russia warns Iraq, Kurds not to destabilize Mi...",moscow reuters russia wednesday warned iraq ku...,1
3,3,WATCH STEVE SCALISE Throw A Strike At The Nati...,house majority whip steve scalise r la threw s...,0
4,4,Trump Will HATE What Stephen Colbert Just Did...,said late show host stephen colbert clearly gi...,0


In [ ]:
# prompt: remove all lines from the dataframe where the text is only whitespaces and drop unneccessary columns title and unnamed from df

import pandas as pd
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

# Assuming df is already loaded and preprocessed as in the provided code

# Remove rows where 'text' column contains only whitespace
df = df[df['text'].str.strip() != ""]

# Drop unnecessary columns
df = df.drop(columns=['title', 'Unnamed: 0'], errors='ignore')

df.head()

,text,label
0,maury perhaps one trashiest shows television t...,0
1,yesterday father one ucla players arrested chi...,0
2,moscow reuters russia wednesday warned iraq ku...,1
3,house majority whip steve scalise r la threw s...,0
4,said late show host stephen colbert clearly gi...,0


## Text cleaning validation

In [ ]:
#check text cleaning

def check_text_cleaning(text):
  """Checks if the text is in lowercase and HTML tags are removed."""
  if not isinstance(text, str):
    return False

  if text.islower() and "<" not in text and ">" not in text:
    return True
  else:
    return False

# Apply the function to the 'text' column and check if all rows are true
cleaned_status = df['text'].apply(check_text_cleaning).all()

print(cleaned_status)

True


Lemmatization for further simplification of the text.

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stem_and_lemmatize(text):
  words = text.split()
  stemmed_words = [stemmer.stem(word) for word in words]
  lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
  return " ".join(lemmatized_words)


df['text'] = df['text'].apply(stem_and_lemmatize)
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,text,label
0,mauri perhap one trashiest show televis today ...,0
1,yesterday father one ucla player arrest china ...,0
2,moscow reuter russia wednesday warn iraq kurd ...,1
3,hous major whip steve scalis r la threw strike...,0
4,said late show host stephen colbert clearli gi...,0


## Validating text preprocessing results

In [ ]:
# prompt: check if stemming and lemmatizing are done correctly

import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# ... (rest of your existing code)

# Improved check_text_cleaning function
def check_text_cleaning(text):
    """Checks if stemming and lemmatization were applied correctly."""
    if not isinstance(text, str):
        return False

    # Check for stemmed words (heuristic)
    words = text.split()
    for word in words:
        if len(word) > 3 and word.endswith("ing") :  # Example heuristic
            return False
    return True

# Apply the function to the 'text' column and check if all rows are true
cleaned_status = df['text'].apply(check_text_cleaning).all()

cleaned_status

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


False

In [ ]:
x = df['text'].values
y = df['label'].values

x.shape, y.shape

((24352,), (24352,))































































































































































































































































































# Splitting dataset into training, testing and validation splits

In [ ]:
# prompt: split the ds text and labels to train eval and test

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Split data into training and temporary sets (80% train, 20% temp)
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)

# Split temporary set into validation and test sets (50% eval, 50% test)
x_eval, x_test, y_eval, y_test = train_test_split(x_temp[:-1], y_temp[:-1], test_size=0.5, random_state=42)

# Now you have x_train, y_train, x_eval, y_eval, x_test, y_test
print("Train data shape:", x_train.shape, y_train.shape)
print("Eval data shape:", x_eval.shape, y_eval.shape)
print("Test data shape:", x_test.shape, y_test.shape)

Train data shape: (19481,) (19481,)
Eval data shape: (2435,) (2435,)
Test data shape: (2435,) (2435,)


# Vectorisation
Counter vectorizer

In [ ]:
# prompt: apply countvectorizer to the x train test and val

vectorizer = CountVectorizer(max_features=5000)

# Fit and transform the training data
x_train_vec = vectorizer.fit_transform(x_train).toarray()

# Transform the validation and test data using the same vectorizer
x_eval_vec = vectorizer.transform(x_eval).toarray()
x_test_vec = vectorizer.transform(x_test).toarray()

print("Train data shape after vectorization:", x_train_vec.shape)
print("Eval data shape after vectorization:", x_eval_vec.shape)
print("Test data shape after vectorization:", x_test_vec.shape)

Train data shape after vectorization: (19481, 5000)
Eval data shape after vectorization: (2435, 5000)
Test data shape after vectorization: (2435, 5000)


In [ ]:
# prompt: use countervectorizer to vectorize the datasets into x_train_cv x_eval_cv and x_test_cv sets

# Assuming x_train, x_eval, and x_test are already defined as in the previous code

# Initialize the CountVectorizer
vectorizer = CountVectorizer(max_features=5000)

# Fit and transform the training data
x_train_cv = vectorizer.fit_transform(x_train)

# Transform the evaluation and test data using the same vocabulary
x_eval_cv = vectorizer.transform(x_eval)
x_test_cv = vectorizer.transform(x_test)

TF-IDF vectorisation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# Fit and transform the training data
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)

# Transform the validation and test data using the same vectorizer
x_eval_tfidf = tfidf_vectorizer.transform(x_eval)
x_test_tfidf = tfidf_vectorizer.transform(x_test)

print("Train data shape after TF-IDF:", x_train_tfidf.shape)
print("Eval data shape after TF-IDF:", x_eval_tfidf.shape)
print("Test data shape after TF-IDF:", x_test_tfidf.shape)

Train data shape after TF-IDF: (19481, 1000)
Eval data shape after TF-IDF: (2435, 1000)
Test data shape after TF-IDF: (2435, 1000)


# Creating and training models
Neural network

In [ ]:
# prompt: create a neurat network and train it on x_train_cv vectors and use x_eval_cv for validation and x_test_cv for testing

import tensorflow as tf

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(x_train_cv.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train_cv, y_train, epochs=10, batch_size=32, validation_data=(x_eval_cv, y_eval))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test_cv, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 1005s 2s/step - accuracy: 0.9047 - loss: 0.2832 - val_accuracy: 0.9733 - val_loss: 0.0982
Epoch 2/10
 51/609 ━━━━━━━━━━━━━━━━━━━━ 15:23 2s/step - accuracy: 0.9790 - loss: 0.0676

In [ ]:
# prompt: generate a classificator neural network with two layer and apply to countervectorized data
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import scipy

# Define the model
model_cv = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(x_train_vec.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

# Compile the model
model_cv.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model using the dense data
model_cv.fit(x_train_vec, y_train, epochs=2, batch_size=32, validation_data=(x_eval_vec, y_eval))

Epoch 1/2
609/609 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9344 - loss: 0.2118 - val_accuracy: 0.9762 - val_loss: 0.0918
Epoch 2/2
609/609 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9902 - loss: 0.0296 - val_accuracy: 0.9737 - val_loss: 0.0950


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model_cv.evaluate(x_test_vec, y_test)
print('Test accuracy with CV:', accuracy)

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9735 - loss: 0.0906
Test accuracy with CV: 0.9737166166305542


In [ ]:
print(x_eval_vec)
print(x_test_vec)

[[0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# prompt: generate a neural network with two layer apply tfidf to it

# Define the model
model_tfidf = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(x_train_tfidf.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

# Compile the model
model_tfidf.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model_tfidf.fit(x_train_tfidf, y_train, epochs=2, batch_size=32, validation_data=(x_eval_tfidf, y_eval))

# Evaluate the model on the test set
loss, accuracy = model_tfidf.evaluate(x_test_tfidf, y_test)
print('Test accuracy with TF-IDF:', accuracy)


Epoch 1/2
609/609 ━━━━━━━━━━━━━━━━━━━━ 477s 781ms/step - accuracy: 0.8812 - loss: 0.2633 - val_accuracy: 0.9639 - val_loss: 0.1190
Epoch 2/2
609/609 ━━━━━━━━━━━━━━━━━━━━ 277s 411ms/step - accuracy: 0.9718 - loss: 0.0867 - val_accuracy: 0.9676 - val_loss: 0.1070
77/77 ━━━━━━━━━━━━━━━━━━━━ 45s 592ms/step - accuracy: 0.9709 - loss: 0.0857
Test accuracy with TF-IDF: 0.9708418846130371


Exporting the trained models into files.

In [ ]:
# prompt: save the model in a file and the vectorizer

import joblib

# Save the model
model_cv.save('fake_news_model.keras')
model_tfidf.save('fake_news_tfidf.keras')



In [ ]:
import joblib
# Save the vectorizer
joblib.dump(vectorizer, 'cv.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf.pkl')

['tfidf.pkl']

# Creating and training models
Logistic regression

In [ ]:
# prompt: create a model with binear regression for classification using the x_train_vec and y_train_vec vectors

### Javitás: logisztikai regresszió használata lineáris regresszió helyett az osztályozáshoz
from sklearn.linear_model import LogisticRegression

# Initialize and train the Logistic Regression model
logreg_model_cv = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence
logreg_model_cv.fit(x_train_vec, y_train)

# Make predictions and evaluate the model
y_pred = logreg_model_cv.predict(x_eval_vec)

In [ ]:
# prompt: create a model with binear regression using the x_train_tfidf and y_train_tfidf vectors and name the model logreg_model_tfidf

# Initialize and train the Logistic Regression model for TF-IDF

### Javitás: logisztikai regresszió használata lineáris regresszió helyett az osztályozáshoz
logreg_model_tfidf = LogisticRegression(max_iter=1000)
logreg_model_tfidf.fit(x_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

Saving the model to a file.

In [ ]:
# prompt: save the two logistic regression models into files

import joblib

# Save the logistic regression models
joblib.dump(logreg_model_cv, 'logreg_model_cv.pkl')
joblib.dump(logreg_model_tfidf, 'logreg_model_tfidf.pkl')

['logreg_model_tfidf.pkl']

# Selecting the best model
Testing every model on the test dataset and finding the best using the accuracy value.

In [ ]:
# prompt: compare all 4 models on the test set (print a table containing the accuracy and loss for every model) and select the best performing one
from sklearn.metrics import accuracy_score # import accuracy_score


## Javitás: a modellek tesztelése és a legjobb megkeresése
# Evaluate all models on the test set
models = {
    'NN_CV': model_cv,
    'NN_TFIDF': model_tfidf,
    'LR_CV': logreg_model_cv,
    'LR_TFIDF': logreg_model_tfidf
}

results = []
for name, model in models.items():
  if name.startswith('NN'):
    x_test_vec = x_test_vec if name == 'NN_CV' else x_test_tfidf
    loss, accuracy = model.evaluate(x_test_vec, y_test, verbose=0)
    results.append({'model': name, 'accuracy': accuracy, 'loss': loss})
  elif name.startswith('LR'):
      x_test_vec = x_test_vec if name == 'LR_CV' else x_test_tfidf
      y_pred = model.predict(x_test_vec)
      accuracy = accuracy_score(y_test, y_pred)
      loss = 1 - accuracy # Placeholder loss for LR
      results.append({'model': name, 'accuracy': accuracy, 'loss':loss})

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Print the table
print(results_df)

# Select the best performing model based on accuracy
best_model_name = results_df.loc[results_df['accuracy'].idxmax(), 'model']
print(f"\nThe best performing model is: {best_model_name}")

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_15" is incompatible with the layer: expected axis -1 of input shape to have value 5000, but received input with shape (None, 1000)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 1000), dtype=float32)
  • training=False
  • mask=None

Title: Fake News Detection using Machine Learning

Introduction (1 minute)

Problem: Fake news is a widespread problem that can have serious consequences.
Solution: We can use machine learning to automatically detect fake news articles.
Approach: We will train a neural network model on a dataset of labeled news articles.
Data Preprocessing (2 minutes)

Data Source: The code uses the GonzaloA/fake_news dataset from Hugging Face Datasets.
Cleaning: The clean_text_advanced function is used to clean the text data:
Lowercasing
Removing punctuation, HTML tags, and extra whitespace
Removing stop words
Stemming and Lemmatization:
stem_and_lemmatize function is used to reduce words to their base forms using Porter Stemmer and WordNet Lemmatizer.
This helps to improve the accuracy of the model.
Verification:
The check_text_cleaning function checks if cleaning and normalization were successful.
Feature Extraction (2 minutes)

CountVectorizer:
Converts text into numerical vectors based on word frequencies.
Creates a vocabulary of the most frequent words.
TF-IDF Vectorizer:
Similar to CountVectorizer, but also considers the importance of words in the entire dataset.
Gives higher weight to words that are frequent in a document but rare in the corpus.
Model Training and Evaluation (3 minutes)

Neural Network:
A simple two-layer neural network is used for classification.
Activation functions: ReLU for hidden layers, softmax for output layer.
Loss function: Sparse categorical cross-entropy.
Optimizer: Adam.
Training:
The model is trained on the training set and validated on the validation set.
The number of epochs and batch size are specified.
Evaluation:
The model's performance is evaluated on the test set using accuracy as a metric.
Saving and Loading the Model (1 minute)

Saving:
The trained model and the vectorizer are saved to files using model.save and joblib.dump.
Loading:
The saved model and vectorizer can be loaded back using tf.keras.models.load_model and joblib.load.
Custom Data Prediction (1 minute)

Input:
The user can input custom text for classification.
Preprocessing:
The input text is preprocessed using the same functions used for training data.
Prediction:
The loaded model is used to predict the class of the input text.
Output:
The predicted class is displayed to the user.
Conclusion

The code demonstrates how to build a simple fake news detection system using machine learning.
Further improvements can be made by experimenting with different models, hyperparameters, and feature engineering techniques.
This project highlights the potential of AI in addressing real-world problems like fake news.